In [1]:
import pandas as pd
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

In [2]:
def fillnan(data, years_c):
    for name in years_c:
        #заменяем троеточия
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))

    return data

def dropempt(data, years_c):
    data = fillnan(data, years_c)
    return data.dropna(thresh=8)

def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if item not in ['Country Name', 'Country Code','Series Code','Series Name']]
    print ('Before clear:', len(data))

    data = dropempt(data, years_c)

    print ('After clear:', len(data))
    return data

In [3]:
def Make_region(code, reg='region'):
    #print code
    if code in data_cnt['name'].values:
        #print code
        name = (data_cnt['name'][data_cnt['name'] == code].index.tolist())[0]
        return data_cnt[reg].at[name]
    else:
        return np.nan

def Sorting(data, reg='region'):
    #if data['Country Name'].get_value(len(data)-5) == np.nan:
    #    data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    if 'Region' not in data.columns:
        data['Region'] = data['Country Name'].progress_apply(lambda x: Make_region(x, reg))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [4]:
df = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8').\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics_by_Wealth_Quintile.xlsx',encoding='utf8'))

data_cnt = pd.read_csv('data/all.csv', encoding='utf8')
df = Clearing(df)
defen = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', sheet_name=1, encoding = 'utf8').\
               append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheet_name=1, encoding='utf8')).\
               append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', sheet_name=1, encoding='utf8'))

Before clear: 398541
After clear: 112610


In [5]:
df = Sorting(df, 'sub-region')

HBox(children=(IntProgress(value=0, max=112610), HTML(value='')))

In [6]:
years_c = [item for ind, item in enumerate(np.array(df.columns)) if item not in ['Region','Country Name', 'Country Code','Series Code','Series Name', 'index']]

In [7]:
df[(df['Series Name'] == 'Population ages 65 and above, total') & (df['Country Code']== 'RUS')][years_c]

,2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016]
81474,19597557.0,19880916.0,19832971.0,19760354.0,19486678.0,19134465.0,18851968.0,18694313.0,18756417.0,18854368.0,18982795.0,19165394.0,19436685.0,19905694.0


In [8]:
df[(df['Series Name'] == 'Population, total') & (df['Country Code']== 'RUS')][years_c].head(1)

,2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016]
23611,144648257.0,144067054.0,143518523.0,143049528.0,142805088.0,142742350.0,142785342.0,142849449.0,142960868.0,143201676.0,143506911.0,143819666.0,144096870.0,144342396.0
